In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1,2"
from vllm import LLM

/root/prime-rl/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO 06-03 14:10:54 [__init__.py:239] Automatically detected platform cuda.


2025-06-03 14:10:56,562	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
from src.zeroband.inference.pipeline import PipelineConfig, patch_model_load

config = PipelineConfig(
    rank=0,
    world_size=2,
    iroh_seed=None,
    iroh_peer_id=None,
    connection_num_retries=3,
)
patch_model_load(config)

06-03 14:10:58 [INFER] [INFO] Patching model init for pp.rank=0 in pp.world_size=2


In [3]:
llm = LLM(
    model="Qwen/Qwen3-0.6B",
    tensor_parallel_size=2,
    max_seq_len_to_capture=16384,
    max_model_len=16384,
    quantization=None,
    enforce_eager=True,
    disable_async_output_proc=True,
    download_dir="/alloc",
    dtype="bfloat16")

INFO 06-03 14:11:05 [config.py:717] This model supports multiple tasks: {'reward', 'score', 'classify', 'embed', 'generate'}. Defaulting to 'generate'.
WARNING 06-03 14:11:05 [arg_utils.py:1658] --disable-async-output-proc is not supported by the V1 Engine. Falling back to V0. We recommend to remove --disable-async-output-proc from your config in favor of the V1 Engine.
INFO 06-03 14:11:05 [config.py:1770] Defaulting to use mp for distributed inference
INFO 06-03 14:11:05 [llm_engine.py:240] Initializing a V0 LLM engine (v0.8.5.post1) with config: model='Qwen/Qwen3-0.6B', speculative_config=None, tokenizer='Qwen/Qwen3-0.6B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=16384, download_dir='/alloc', load_format=LoadFormat.AUTO, tensor_parallel_size=2, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=True, kv_cache_d

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00, 11.35it/s]


INFO 06-03 14:11:09 [loader.py:458] Loading weights took 0.10 seconds
INFO 06-03 14:11:10 [model_runner.py:1140] Model loading took 0.3628 GiB and 0.339777 seconds


(VllmWorkerProcess pid=357794) INFO 06-03 14:11:10 [weight_utils.py:315] No model.safetensors.index.json found in remote.
(VllmWorkerProcess pid=357794) INFO 06-03 14:11:10 [loader.py:458] Loading weights took 0.10 seconds
(VllmWorkerProcess pid=357794) INFO 06-03 14:11:10 [model_runner.py:1140] Model loading took 0.3628 GiB and 0.694652 seconds
(VllmWorkerProcess pid=357794) Input
(VllmWorkerProcess pid=357794) hidden_states[-1][-5:]: [0.0546875, 0.00909423828125, 0.011962890625, -0.01055908203125, 0.0159912109375]
(VllmWorkerProcess pid=357794) residual is None
(VllmWorkerProcess pid=357794) Layer 0
(VllmWorkerProcess pid=357794) hidden_states[-1][-5:]: [-0.044921875, -0.0185546875, 0.06103515625, -0.0830078125, -0.0274658203125]
(VllmWorkerProcess pid=357794) residual[-1][-5:]: [0.1123046875, 0.11474609375, 0.0732421875, 0.1279296875, -0.0001220703125]
(VllmWorkerProcess pid=357794) Layer 1
(VllmWorkerProcess pid=357794) hidden_states[-1][-5:]: [0.021728515625, 0.1611328125, -0.2128

In [4]:
from vllm import SamplingParams

sampling_params = SamplingParams(
    max_tokens=1,
    temperature=0.7,
    top_p=0.9,
    top_k=40,
    seed=42,
)
request_outputs = llm.generate("Hello, world!", sampling_params)
print(request_outputs[0].outputs[0].text)

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  4.76it/s, est. speed input: 19.10 toks/s, output: 4.77 toks/s]

(VllmWorkerProcess pid=357794) Input
(VllmWorkerProcess pid=357794) hidden_states[-1][-5:]: [0.0546875, 0.00909423828125, 0.011962890625, -0.01055908203125, 0.0159912109375]
(VllmWorkerProcess pid=357794) residual is None
(VllmWorkerProcess pid=357794) Layer 0
(VllmWorkerProcess pid=357794) hidden_states[-1][-5:]: [0.0255126953125, -0.0361328125, -0.057861328125, -0.0111083984375, 0.0341796875]
(VllmWorkerProcess pid=357794) residual[-1][-5:]: [-0.0234375, 0.2109375, 0.01953125, 0.09765625, 0.06298828125]
(VllmWorkerProcess pid=357794) Layer 1
(VllmWorkerProcess pid=357794) hidden_states[-1][-5:]: [0.2216796875, -0.08544921875, -0.022705078125, -0.029541015625, 0.01611328125]
(VllmWorkerProcess pid=357794) residual[-1][-5:]: [-0.04931640625, 0.005859375, 0.2001953125, 0.11181640625, 0.06591796875]
(VllmWorkerProcess pid=357794) Layer 2
(VllmWorkerProcess pid=357794) hidden_states[-1][-5:]: [0.0009765625, 0.25390625, -0.30859375, 0.12109375, 0.1982421875]
(VllmWorkerProcess pid=357794) 

Input
hidden_states[-1][-5:]: [0.0546875, 0.00909423828125, 0.011962890625, -0.01055908203125, 0.0159912109375]
residual is None
Layer 0
hidden_states[-1][-5:]: [0.0255126953125, -0.0361328125, -0.057861328125, -0.0111083984375, 0.0341796875]
residual[-1][-5:]: [-0.0234375, 0.2109375, 0.01953125, 0.09765625, 0.06298828125]
Layer 1
hidden_states[-1][-5:]: [0.2216796875, -0.08544921875, -0.022705078125, -0.029541015625, 0.01611328125]
residual[-1][-5:]: [-0.04931640625, 0.005859375, 0.2001953125, 0.11181640625, 0.06591796875]
Layer 2
hidden_states[-1][-5:]: [0.0009765625, 0.25390625, -0.30859375, 0.12109375, 0.1982421875]
residual[-1][-5:]: [0.0478515625, -0.07373046875, 0.3046875, -0.009765625, 0.2490234375]
Layer 3
hidden_states[-1][-5:]: [0.12890625, -0.0693359375, -0.2001953125, 0.2109375, 0.18359375]
residual[-1][-5:]: [0.057861328125, 0.2890625, 0.1240234375, 0.08544921875, 0.2236328125]
Layer 4
hidden_states[-1][-5:]: [-0.018798828125, 0.1533203125, -0.181640625, 0.236328125, -0.0